# Error 500 traceback for debugging
In this code we try to catch the exceptions caused by an error on the website in order to save a file with the information and try to repair the error with that info. 

In [ ]:
import json
import sys
import os
import traceback
from django.utils.deprecation import MiddlewareMixin
from django.utils.timezone import now


class ErrorLoggingMiddleware(MiddlewareMixin):
    def process_exception(self, request, exception):
        traceback_info = "".join(traceback.format_exception(*sys.exc_info()))
        user_name = request.user.username if request.user.is_authenticated else "Anonymous"
        error_data = {
            "timestamp": now().isoformat(),
            "url": request.path,
            "method": request.method,
            "error": str(exception),
            "traceback": traceback_info,
            "user": user_name,
        }
        file_path = f"errors/{now().strftime('%Y-%m-%d_%H-%M-%S')}_{user_name}.log"
        try:
            with open(file_path, "w") as f:
                f.write(traceback_info)
        except Exception as file_error:
            print(f"Failed to save traceback to file: {file_error}")
        json_data = json.dumps({"Error": error_data["error"], "Url": error_data["url"]})
        data_loaded = json.loads(json_data)
        print(f"Data loaded: {data_loaded}")
        if "delete_file" in request.POST:
            if os.path.exists(file_path):
                os.remove(file_path)
            